In [ ]:
import findspark
findspark.add_packages("graphframes:graphframes:0.8.0-spark3.0-s_2.12")
findspark.init()
import pyspark
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-0.90.jar, xgboost4j-spark-0.90.jar pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS']=" --master local[2] pyspark-shell"
sc = pyspark.SparkContext(appName = "MyAPP")
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)
sc.addPyFile('/home/msbd5003/graphframes-0.8.0-spark3.0-s_2.12.jar')

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd
import numpy as np
from datetime import date
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType


In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
dataset1 = spark.read.csv('data/dataset1.csv',header=True, inferSchema=True)
dataset2 = spark.read.csv('data/dataset2.csv',header=True, inferSchema=True)
#dataset3 is for competition
#dataset3 = spark.read.csv('data/dataset3.csv',header=True, inferSchema=True)

print(dataset1.count())
print(dataset2.count())
print(dataset1.columns)
print(len(dataset1.columns))
print(len(dataset2.columns))

In [ ]:
minus_to_zero = udf(lambda x : 0 if x == -1 else x)

#for regression, label only 1 or 0
dataset1 = dataset1.withColumn('Label',minus_to_zero(col('Label'))) 
dataset2 = dataset2.withColumn('Label',minus_to_zero(col('Label'))) 
print(dataset1.count())
print(dataset2.count())

In [ ]:


def cal_discount_rate(s):
    s =str(s)
    s = s.split(':')
    
    if len(s)==1:
        #print(s[0])
        return float(s[0])
    else:
        return float(1.0-float(s[1])/float(s[0]))
    
calc_discount_rate = udf(lambda x : cal_discount_rate(x))

check_not_float = udf(lambda x : x if type(x) == float else float(x))
    
dataset_train = dataset2.union(dataset1)
#dataset_train = dataset_train.replace
dataset_train = dataset_train.withColumn('Distance',col('Distance').cast(IntegerType()))
dataset_train.select('Discount_rate').show()
dataset_train = dataset_train.withColumn('Discount_rate',calc_discount_rate(col('Discount_rate')))
dataset_train = dataset_train.withColumn('Discount_rate',check_not_float(col('Discount_rate')))

from pyspark.sql.types import DoubleType
dataset_train = dataset_train.withColumn('Discount_rate',col('Discount_rate').cast(DoubleType()))
dataset_train = dataset_train.select([col(c).cast(DoubleType()) for c in dataset_train.columns])

dataset_train.select('Discount_rate').show()
dataset_train.printSchema()
print(dataset_train.count())
print(len(dataset_train.columns))



In [ ]:
drop_list = ['User_id']
dataset_train = dataset_train.drop(*drop_list)
print(dataset_train.count())
print(dataset_train.columns)

In [ ]:
col_list = ['Date_received', 'Discount_rate', 'Distance', 'Day_of_mouth', 'Days_distance', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'user_coupon_transfer_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Label_user_merchant_coupon_count', 'Label_merchant_coupon_count', 'Coupon_count_later', 'Month_Receive_same_coupon_count', 'Month_Receive_all_coupon_count', 'Month_same_coupon_lastone', 'Month_same_coupon_firstone', 'Day_Receive_all_coupon_count', 'Day_Receive_same_coupon_count', 'Is_weekend', 'Weekday1', 'Weekday2', 'Weekday3', 'Weekday4', 'Weekday5', 'Weekday6', 'Weekday7']
col_list_not_rate_distance = ['Date_received','Day_of_mouth', 'Days_distance', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'user_coupon_transfer_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Label_user_merchant_coupon_count', 'Label_merchant_coupon_count', 'Coupon_count_later', 'Month_Receive_same_coupon_count', 'Month_Receive_all_coupon_count', 'Month_same_coupon_lastone', 'Month_same_coupon_firstone', 'Day_Receive_all_coupon_count', 'Day_Receive_same_coupon_count', 'Is_weekend', 'Weekday1', 'Weekday2', 'Weekday3', 'Weekday4', 'Weekday5', 'Weekday6', 'Weekday7']

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


assembler = VectorAssembler(inputCols=col_list, outputCol = 'features', handleInvalid='skip')
#assembler = VectorAssembler(inputCols=col_list, outputCol = 'features', handleInvalid='keep')
#assembler = VectorAssembler(inputCols=col_list_not_rate, outputCol = 'features')
#print(dataset_train.count())
dataset_train = assembler.transform(dataset_train)
dataset_train = dataset_train.drop(*col_list)
print(dataset_train.count())
dataset_train.printSchema()
dataset_train.show()
#dataset_train_ass.select('features').show(10,False)

In [ ]:

#train validation test 0.8:0.1:0.1  seed 1
#rdd_dataset = dataset_train.randomSplit([0.8,0.1,0.1],1)
rdd_dataset = dataset_train.randomSplit([0.9,0.1],1)
#dataset3 is testing data set
#feature_list = rdd_dataset[0].columns
#feature_list.remove('Label')
#rdd_dataset[0] = rdd_dataset[0].withColumn('features', concat_ws(",",array(*feature_list))).drop(*feature_list)
#print(rdd_dataset[0].count())
#rdd_dataset[0].show(5)


In [ ]:
from pyspark.ml import Pipeline



spark.sparkContext.addPyFile("sparkxgb.zip") # read xgboost pyspark client lib
from sparkxgb import XGBoostClassifier

xgb = XGBoostClassifier(
    objective="reg:logistic",
    maxDepth=5,
    missing=float(0.0),
    featuresCol="features", 
    labelCol="Label", 
)
#rf = RandomForestRegressor(numTrees=15,maxDepth=6,labelCol = 'Label')
#rf.explainParams()
#xgb.setSeed(1)

In [ ]:
rdd_dataset[0].printSchema()
model1 = xgb.transform(rdd_dataset[0])
#model1.getBootstrap()
#model1.setLeafCol("leafID")

In [ ]:
threshold = udf(lambda x : 1 if x>=0.5 else 0)

prediction_v = model1.fit(rdd_dataset[1])

prediction_v.printSchema()
print(prediction_v.take(1))
prediction_v = prediction_v.withColumn('prediction',threshold(col('prediction')))
numSuccesses = prediction_v.where('Label == prediction').count()

print(prediction_v.take(1))

numInspections = prediction_v.count()

print ("There were %d inspections and there were %d successful predictions" % (numInspections, numSuccesses))
print("This is a %d%% success rate" % (float(numSuccesses) / float(numInspections) * 100))

#prediction_t = model1.transform(rdd_dataset[2])
#numSuccesses = prediction_t.where('Label == prediction').count()
#numInspections = prediction_t.count()
#print ("There were %d inspections and there were %d successful predictions" % (numInspections, numSuccesses))
#print("This is a %d%% success rate" % (float(numSuccesses) / float(numInspections) * 100))